# Text Classification Using Transformer Networks (BERT)

Some initialization:

In [1]:
!pip3 install datasets
!pip3 install transformers
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 46.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

In [3]:
def read_dota_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["text","target"]]

In [79]:
labels = open('classes.txt').read().splitlines()
data = read_dota_data('1/tagged-data.csv')
print(labels)
data = data.rename(columns={"target": "label"})
data

['0', '1', '2']


,text,label
0,COMMEND ME TY,0
1,sorry nex,0
2,what is the best soup?,0
3,man that silence on axe,0
4,not coming into play,0
...,...,...
3262,"wt?f?asfU JGOFIDLK,YH",1
3263,you must really suck,2
3264,YOU HAVE IDIOT PLAYER,2
3265,SUPER IDIOT,2


In [80]:
from sklearn.model_selection import train_test_split

train_df, eval_and_test_df = train_test_split(data, train_size=0.8)
eval_df, test_df = train_test_split(eval_and_test_df, train_size=0.5)
train_df.reset_index(inplace=True, drop=True)
eval_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

print(f'train rows: {len(train_df.index):,}')
print(f'eval rows: {len(eval_df.index):,}')
print(f'test rows: {len(test_df.index):,}')

train rows: 2,613
eval rows: 327
test rows: 327


In [81]:
eval_df

,text,label
0,btch,2
1,him go to school,0
2,best carry,0
3,comend tusk,0
4,can we end PLS BRO,0
...,...,...
322,this pudge so fucking stupid,2
323,second try lucky,0
324,u know that,0
325,it took my shaman 12 min to ward my lane,0


In [82]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_df)
ds['validation'] = Dataset.from_pandas(eval_df)
ds['test'] = Dataset.from_pandas(test_df)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2613
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 327
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 327
    })
})

Tokenize the texts:

In [83]:
from transformers import AutoTokenizer

transformer_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

In [84]:
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True)

train_ds = ds['train'].map(
    tokenize, 
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
eval_ds = ds['validation'].map(
    tokenize,
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
train_ds.to_pandas()

Map:   0%|          | 0/2613 [00:00<?, ? examples/s]

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 1128, 1132, 1198, 1112, 6870, 1183, 1112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,0,"[101, 4877, 2286, 185, 3447, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
2,1,"[101, 1831, 28000, 1158, 1105, 1505, 175, 4419...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,0,"[101, 142, 16769, 7462, 22983, 2036, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
4,0,"[101, 1842, 1183, 136, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
...,...,...,...,...
2608,0,"[101, 192, 1204, 2087, 1110, 1142, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
2609,0,"[101, 7642, 5773, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
2610,0,"[101, 2959, 3713, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
2611,0,"[101, 1160, 12261, 188, 18628, 4206, 23684, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"


Create the transformer model:

In [85]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [86]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    transformer_name,
    num_labels=len(labels),
    # num_labels=2
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config)
)

Create the trainer object and train:

In [87]:
from transformers import TrainingArguments

num_epochs = 2
batch_size = 24
weight_decay = 0.01
model_name = f'{transformer_name}-sequence-classification'

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    weight_decay=weight_decay,
)

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [88]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [89]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

/tmp/ipykernel_1261/485909320.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [90]:
trainer.train()

{'eval_loss': 0.3257763981819153, 'eval_accuracy': 0.8715596330275229, 'eval_runtime': 0.6209, 'eval_samples_per_second': 526.629, 'eval_steps_per_second': 22.547, 'epoch': 1.0}
{'eval_loss': 0.32719868421554565, 'eval_accuracy': 0.8807339449541285, 'eval_runtime': 0.5515, 'eval_samples_per_second': 592.891, 'eval_steps_per_second': 25.384, 'epoch': 2.0}
{'train_runtime': 43.2016, 'train_samples_per_second': 120.968, 'train_steps_per_second': 5.046, 'train_loss': 0.43179604766565727, 'epoch': 2.0}


TrainOutput(global_step=218, training_loss=0.43179604766565727, metrics={'train_runtime': 43.2016, 'train_samples_per_second': 120.968, 'train_steps_per_second': 5.046, 'train_loss': 0.43179604766565727, 'epoch': 2.0})

Evaluate on the test partition:

In [94]:
test_ds = ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds.to_pandas()

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 8147, 1268, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
1,0,"[101, 1185, 189, 1204, 4268, 131, 113, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
2,2,"[101, 9367, 1103, 1342, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
3,0,"[101, 18732, 25290, 11680, 2137, 22157, 157, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,1,"[101, 1142, 1110, 1136, 1139, 8750, 6088, 1120...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...
322,0,"[101, 5671, 1159, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
323,1,"[101, 1274, 1204, 1400, 1155, 8750, 1285, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
324,0,"[101, 190, 1274, 1204, 1444, 4035, 23655, 2737...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
325,0,"[101, 188, 1616, 14166, 1568, 1591, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"


In [95]:
output = trainer.predict(test_ds)
output

PredictionOutput(predictions=array([[ 4.580799  , -2.868516  , -2.6602206 ],
       [ 4.189622  , -2.6161432 , -2.344411  ],
       [-2.5880606 ,  3.3795323 , -0.8629769 ],
       [ 3.7426946 , -2.9373856 , -2.0525022 ],
       [-2.2710679 ,  2.836999  , -0.9064354 ],
       [-2.9694412 ,  2.9193392 , -0.19312295],
       [ 4.3639402 , -3.2938905 , -2.410439  ],
       [-2.7103593 , -0.68810624,  2.5079951 ],
       [ 4.7041464 , -3.598425  , -1.9860016 ],
       [ 3.8959372 , -2.6260152 , -2.0801125 ],
       [-0.28866628, -1.4368327 ,  0.53579104],
       [ 3.7710135 , -3.0539846 , -1.1221324 ],
       [-3.5330162 ,  0.04097487,  2.9096947 ],
       [ 4.0402822 , -3.3952386 , -1.7955227 ],
       [ 4.7021914 , -3.339149  , -2.3963218 ],
       [-3.2164176 ,  2.984665  , -0.2384363 ],
       [ 4.509323  , -3.897624  , -2.073632  ],
       [ 3.774676  , -3.4613605 , -1.9652206 ],
       [-3.422785  , -0.53170866,  3.1443765 ],
       [-3.1404371 , -0.8161251 ,  2.8167133 ],
       [-3.

In [97]:
from sklearn.metrics import classification_report

y_true = output.label_ids
y_pred = np.argmax(output.predictions, axis=-1)
target_names = ["not toxic","kinda toxic","toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.94      0.93      0.94       180
 kinda toxic       0.73      0.75      0.74        48
       toxic       0.82      0.82      0.82        99

    accuracy                           0.87       327
   macro avg       0.83      0.83      0.83       327
weighted avg       0.87      0.87      0.87       327

